<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pairwise-association-mining" data-toc-modified-id="Pairwise-association-mining-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pairwise association mining</a></span><ul class="toc-item"><li><span><a href="#Sets-or-lists-of-annotation-terms-per-protein" data-toc-modified-id="Sets-or-lists-of-annotation-terms-per-protein-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Sets or lists of annotation terms per protein</a></span></li><li><span><a href="#Generating-all-possible-pairwise-combinations-(that-are-present)" data-toc-modified-id="Generating-all-possible-pairwise-combinations-(that-are-present)-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generating all possible pairwise combinations (that are present)</a></span></li><li><span><a href="#Create-all-possible-pairs" data-toc-modified-id="Create-all-possible-pairs-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Create all possible pairs</a></span></li><li><span><a href="#Merging-annotations" data-toc-modified-id="Merging-annotations-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Merging annotations</a></span></li><li><span><a href="#Propagate-pairs" data-toc-modified-id="Propagate-pairs-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Propagate pairs</a></span><ul class="toc-item"><li><span><a href="#Store-propagated-pairs-as-sets" data-toc-modified-id="Store-propagated-pairs-as-sets-1.5.1"><span class="toc-item-num">1.5.1&nbsp;&nbsp;</span>Store propagated pairs as sets</a></span></li><li><span><a href="#Store-propagated-pairs-as-lists" data-toc-modified-id="Store-propagated-pairs-as-lists-1.5.2"><span class="toc-item-num">1.5.2&nbsp;&nbsp;</span>Store propagated pairs as lists</a></span></li><li><span><a href="#Store-propagated-pairs-as-sorted-lists" data-toc-modified-id="Store-propagated-pairs-as-sorted-lists-1.5.3"><span class="toc-item-num">1.5.3&nbsp;&nbsp;</span>Store propagated pairs as sorted lists</a></span></li></ul></li><li><span><a href="#Propagated-pairs-dictionary-memory-requirements" data-toc-modified-id="Propagated-pairs-dictionary-memory-requirements-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Propagated pairs dictionary memory requirements</a></span></li><li><span><a href="#Pairwise-assoc" data-toc-modified-id="Pairwise-assoc-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Pairwise assoc</a></span><ul class="toc-item"><li><span><a href="#1-pre-compute-propagated-pairs" data-toc-modified-id="1-pre-compute-propagated-pairs-1.7.1"><span class="toc-item-num">1.7.1&nbsp;&nbsp;</span>1 pre-compute propagated pairs</a></span><ul class="toc-item"><li><span><a href="#Pre-compute-+-pairwise_dict-with-sets-+-no-convert-to-set-during-loop" data-toc-modified-id="Pre-compute-+-pairwise_dict-with-sets-+-no-convert-to-set-during-loop-1.7.1.1"><span class="toc-item-num">1.7.1.1&nbsp;&nbsp;</span>Pre-compute + pairwise_dict with sets + no convert to set during loop</a></span></li><li><span><a href="#Pre-compute-+-pairwise_dict-with-lists-+-convert-to-set-during-loop" data-toc-modified-id="Pre-compute-+-pairwise_dict-with-lists-+-convert-to-set-during-loop-1.7.1.2"><span class="toc-item-num">1.7.1.2&nbsp;&nbsp;</span>Pre-compute + pairwise_dict with lists + convert to set during loop</a></span></li></ul></li><li><span><a href="#3-on-the-fly-calculation-of-propagated-pairs" data-toc-modified-id="3-on-the-fly-calculation-of-propagated-pairs-1.7.2"><span class="toc-item-num">1.7.2&nbsp;&nbsp;</span>3 on the fly calculation of propagated pairs</a></span></li><li><span><a href="#Generator" data-toc-modified-id="Generator-1.7.3"><span class="toc-item-num">1.7.3&nbsp;&nbsp;</span>Generator</a></span></li></ul></li><li><span><a href="#Effect-of-conditional-inside-versus-outside-loops" data-toc-modified-id="Effect-of-conditional-inside-versus-outside-loops-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Effect of conditional inside versus outside loops</a></span></li></ul></li></ul></div>

# Pairwise association mining

In [3]:
import ast
import numpy as np
import pandas as pd
import itertools
import sys

from pathlib import Path

from goscripts import obo_tools
from phppipy.mining import pairwise

import os
os.chdir('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis')
os.getcwd()

'/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis'

In [4]:
# https://goshippo.com/blog/measure-real-size-any-python-object/

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [5]:
input_file = "data/interim//35237/ppi_data/ppi-annotations.tsv"
obo_file = "data/raw/go_data/go.obo"

ppi_file = Path(input_file)
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)
print('PPIs were imported from {}\n'.format(ppi_file))

PPIs were imported from data/interim/35237/ppi_data/ppi-annotations.tsv



In [6]:
for i in ['GO_xref_A', 'GO_xref_B', 'interpro_xref_A', 'interpro_xref_B']:
    ppi_df.loc[~ppi_df[i].isnull(), i] = ppi_df.loc[~ppi_df[i].isnull(),
                                                    i].map(ast.literal_eval)
    
merged_annotations_A = pairwise.merge_annotations(
    ppi_df, ['GO_xref_A', 'interpro_xref_A'])
labeled_merged_annotations_A = pairwise.add_hp_label(merged_annotations_A, 'h')
ppi_df['annotations_A'] = labeled_merged_annotations_A

merged_annotations_B = pairwise.merge_annotations(
    ppi_df, ['GO_xref_B', 'interpro_xref_B'])
labeled_merged_annotations_B = pairwise.add_hp_label(merged_annotations_B, 'p')
ppi_df['annotations_B'] = labeled_merged_annotations_B

In [7]:
ppi_df.columns

Index(['aliases_A', 'aliases_B', 'alt_identifiers_A', 'alt_identifiers_B',
       'author', 'confidence_score', 'detection_method', 'display_id_A',
       'display_id_B', 'interaction_identifiers', 'interaction_type', 'origin',
       'pathogen', 'publication', 'source_database_ids', 'taxid_A', 'taxid_B',
       'xref_A', 'xref_B', 'taxid_name_A', 'taxid_name_B',
       'xref_partners_sorted', 'GO_xref_A', 'GO_xref_B', 'interpro_xref_A',
       'interpro_xref_B', 'annotations_A', 'annotations_B'],
      dtype='object')

In [8]:
# create GO dictionary for propagation
obo_path = Path(obo_file)
go_dict = obo_tools.importOBO(obo_path, ignore_part_of=False)
obo_tools.buildGOtree(
    go_dict, root_nodes=['GO:0008150', 'GO:0005575', 'GO:0003674'])

Retrieved 47179 GO terms from /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/raw/go_data/go.obo 

Adding secondary GO identifiers...



In [9]:
columns = ['annotations_A', 'annotations_B']
column_A = columns[0]
column_B = columns[1]

## Sets or lists of annotation terms per protein

Almost a 10x difference in size of sets compared to lists.

In [8]:
get_size(ppi_df['annotations_A'])

27436061

In [9]:
get_size(ppi_df['annotations_A'].map(list))

5521869

In [10]:
get_size(ppi_df['GO_xref_A'])

23373395

In [11]:
%timeit ppi_df['GO_xref_A'].map(list)
%timeit ppi_df['GO_xref_A'].apply(list)

18.1 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
17.9 ms ± 777 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
get_size(ppi_df['GO_xref_A'].map(list))

4918089

## Generating all possible pairwise combinations (that are present)

The cartesian product cannot contain duplicates of the form `(GO:0016197, GO:0055036) vs (GO:0055036, GO:0016197)`. 

Consequently, calling `set()` on the `itertools.product()` output is not required. A sort can optionally be preformed to equalise different rows.

Storing sets inside a pandas dataframe increases the memory requirements, so a list is prefered.

In [13]:
all(ppi_df[columns].apply(lambda x: sorted(list(set([i for i in itertools.product(x[column_A], x[column_B])]))),axis=1) == ppi_df[columns].apply(lambda x: sorted(list([i for i in itertools.product(x[column_A], x[column_B])])),axis=1))

True

In [14]:
get_size(ppi_df[columns].apply(lambda x: set([i for i in itertools.product(x[column_A], x[column_B])]),axis=1))

226787095

In [15]:
get_size(ppi_df[columns].apply(lambda x: [i for i in itertools.product(x[column_A], x[column_B])],axis=1))

37543503

Sorting has a considerable time impact.

In [16]:
%timeit ppi_df[columns].apply(lambda x: [i for i in itertools.product(x[column_A], x[column_B])],axis=1)

806 ms ± 37.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit ppi_df[columns].apply(lambda x: sorted([i for i in itertools.product(x[column_A], x[column_B])]),axis=1)

2.16 s ± 82.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The cartesian product operates equally fast on lists and sets of annotation terms (likely due to the limited size per row).

In [10]:
ppi_df['annotations_A_list'] = ppi_df['annotations_A'].map(list)
ppi_df['annotations_B_list'] = ppi_df['annotations_B'].map(list)
%timeit ppi_df[columns].apply(lambda x: set([i for i in itertools.product(x[column_A], x[column_B])]),axis=1)
%timeit ppi_df[['annotations_A_list', 'annotations_B_list']].apply(lambda x: set([i for i in itertools.product(x['annotations_A_list'], x['annotations_B_list'])]),axis=1)

1.36 s ± 49.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.35 s ± 87.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
ppi_df['annotations_A'][0]

{'h@GO:0000166',
 'h@GO:0003723',
 'h@GO:0005515',
 'h@GO:0005524',
 'h@GO:0005737',
 'h@GO:0005829',
 'h@GO:0005832',
 'h@GO:0005874',
 'h@GO:0006457',
 'h@GO:0006458',
 'h@GO:0032212',
 'h@GO:0050821',
 'h@GO:0051082',
 'h@GO:0061077',
 'h@GO:0070062',
 'h@GO:0071987',
 'h@GO:1904851',
 'h@GO:1904871',
 'h@GO:1904874',
 'h@IPR002194',
 'h@IPR002423',
 'h@IPR012722',
 'h@IPR017998',
 'h@IPR027409',
 'h@IPR027410',
 'h@IPR027413'}

In [20]:
ppi_df['annotations_A_list'][0]

['h@GO:0071987',
 'h@GO:1904874',
 'h@GO:0005737',
 'h@IPR017998',
 'h@IPR027413',
 'h@GO:0005524',
 'h@GO:0070062',
 'h@GO:0032212',
 'h@GO:0006457',
 'h@GO:0006458',
 'h@GO:0005832',
 'h@GO:0005829',
 'h@IPR002423',
 'h@GO:1904851',
 'h@GO:0000166',
 'h@GO:1904871',
 'h@IPR012722',
 'h@GO:0050821',
 'h@IPR002194',
 'h@GO:0005874',
 'h@IPR027410',
 'h@GO:0003723',
 'h@GO:0005515',
 'h@GO:0061077',
 'h@IPR027409',
 'h@GO:0051082']

In [21]:
list(itertools.product(ppi_df['annotations_A_list'][0], ppi_df['annotations_B_list'][0]))

[('h@GO:0071987', 'p@IPR036157'),
 ('h@GO:0071987', 'p@GO:0016021'),
 ('h@GO:0071987', 'p@GO:0016020'),
 ('h@GO:0071987', 'p@IPR029054'),
 ('h@GO:1904874', 'p@IPR036157'),
 ('h@GO:1904874', 'p@GO:0016021'),
 ('h@GO:1904874', 'p@GO:0016020'),
 ('h@GO:1904874', 'p@IPR029054'),
 ('h@GO:0005737', 'p@IPR036157'),
 ('h@GO:0005737', 'p@GO:0016021'),
 ('h@GO:0005737', 'p@GO:0016020'),
 ('h@GO:0005737', 'p@IPR029054'),
 ('h@IPR017998', 'p@IPR036157'),
 ('h@IPR017998', 'p@GO:0016021'),
 ('h@IPR017998', 'p@GO:0016020'),
 ('h@IPR017998', 'p@IPR029054'),
 ('h@IPR027413', 'p@IPR036157'),
 ('h@IPR027413', 'p@GO:0016021'),
 ('h@IPR027413', 'p@GO:0016020'),
 ('h@IPR027413', 'p@IPR029054'),
 ('h@GO:0005524', 'p@IPR036157'),
 ('h@GO:0005524', 'p@GO:0016021'),
 ('h@GO:0005524', 'p@GO:0016020'),
 ('h@GO:0005524', 'p@IPR029054'),
 ('h@GO:0070062', 'p@IPR036157'),
 ('h@GO:0070062', 'p@GO:0016021'),
 ('h@GO:0070062', 'p@GO:0016020'),
 ('h@GO:0070062', 'p@IPR029054'),
 ('h@GO:0032212', 'p@IPR036157'),
 ('h@GO:

In [22]:
columns

['annotations_A', 'annotations_B']

Not calling `set()` on the final result also gives a (very) small speedup.

In [23]:
%timeit ppi_df[columns].apply(lambda x: set([i for i in itertools.product(x[column_A], x[column_B])]),axis=1)
%timeit ppi_df[columns].apply(lambda x: [i for i in itertools.product(x[column_A], x[column_B])],axis=1)

1.11 s ± 34.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
842 ms ± 37.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
ppi_df['annotation_pairs_set'] = ppi_df[columns].apply(lambda x: set([i for i in itertools.product(x[column_A], x[column_B])]),axis=1)
ppi_df['annotation_pairs'] = ppi_df[columns].apply(lambda x: [i for i in itertools.product(x[column_A], x[column_B])],axis=1)

In [25]:
ppi_df['annotation_pairs']

0        [(h@GO:0071987, p@IPR036157), (h@GO:0071987, p...
1        [(h@GO:0046872, p@GO:0008270), (h@GO:0046872, ...
2        [(h@GO:0016779, p@GO:0003678), (h@GO:0016779, ...
3        [(h@GO:0042995, p@IPR014900), (h@GO:0042995, p...
4        [(h@GO:0005681, p@IPR004966), (h@GO:0005681, p...
5        [(h@GO:0005681, p@IPR014900), (h@GO:0005681, p...
6        [(h@GO:0045444, p@GO:0006260), (h@GO:0045444, ...
7        [(h@GO:0006357, p@GO:0006260), (h@GO:0006357, ...
8        [(h@GO:0045444, p@GO:0003678), (h@GO:0045444, ...
9        [(h@GO:0042405, p@GO:0006260), (h@GO:0042405, ...
10       [(h@GO:0006357, p@GO:0003678), (h@GO:0006357, ...
11       [(h@GO:0033683, p@GO:0006260), (h@GO:0033683, ...
12       [(h@GO:0016779, p@GO:0006260), (h@GO:0016779, ...
13       [(h@GO:0008360, p@GO:0039526), (h@GO:0008360, ...
14       [(h@IPR012151, p@GO:0039526), (h@IPR012151, p@...
15       [(h@GO:0016032, p@GO:0006260), (h@GO:0016032, ...
16       [(h@GO:0017166, p@GO:0039526), (h@GO:0017166, .

Add all present annotation term tuples to one list.

This code must join all tuples across all rows and remove duplicates.

In [26]:
%timeit set(itertools.chain.from_iterable(ppi_df['annotation_pairs']))

855 ms ± 54.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit set(itertools.chain.from_iterable(ppi_df['annotation_pairs'].values))

772 ms ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit set().union(*ppi_df['annotation_pairs'])

778 ms ± 44.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The `union(*column)` operation will always result in the same set, regardless of initial order (as long as the tuples themselves are sorted h@-p@).

In [29]:
a = ppi_df.loc[0:10, 'annotation_pairs'].copy()
a.loc[1] = a.loc[0]
print(a.loc[0])

[('h@GO:0071987', 'p@IPR036157'), ('h@GO:0071987', 'p@GO:0016021'), ('h@GO:0071987', 'p@GO:0016020'), ('h@GO:0071987', 'p@IPR029054'), ('h@GO:1904874', 'p@IPR036157'), ('h@GO:1904874', 'p@GO:0016021'), ('h@GO:1904874', 'p@GO:0016020'), ('h@GO:1904874', 'p@IPR029054'), ('h@GO:0005737', 'p@IPR036157'), ('h@GO:0005737', 'p@GO:0016021'), ('h@GO:0005737', 'p@GO:0016020'), ('h@GO:0005737', 'p@IPR029054'), ('h@IPR017998', 'p@IPR036157'), ('h@IPR017998', 'p@GO:0016021'), ('h@IPR017998', 'p@GO:0016020'), ('h@IPR017998', 'p@IPR029054'), ('h@IPR027413', 'p@IPR036157'), ('h@IPR027413', 'p@GO:0016021'), ('h@IPR027413', 'p@GO:0016020'), ('h@IPR027413', 'p@IPR029054'), ('h@GO:0005524', 'p@IPR036157'), ('h@GO:0005524', 'p@GO:0016021'), ('h@GO:0005524', 'p@GO:0016020'), ('h@GO:0005524', 'p@IPR029054'), ('h@GO:0070062', 'p@IPR036157'), ('h@GO:0070062', 'p@GO:0016021'), ('h@GO:0070062', 'p@GO:0016020'), ('h@GO:0070062', 'p@IPR029054'), ('h@GO:0032212', 'p@IPR036157'), ('h@GO:0032212', 'p@GO:0016021'), ('

In [30]:
b = set().union(*a)

In [31]:
len(b)

3242

In [32]:
a2 = ppi_df.loc[0:10, 'annotation_pairs'].copy()
a2.loc[1] = a2.loc[0]
a2.loc[0] = a2.loc[0][::-1]
print(a2.loc[0])

[('h@GO:0051082', 'p@IPR029054'), ('h@GO:0051082', 'p@GO:0016020'), ('h@GO:0051082', 'p@GO:0016021'), ('h@GO:0051082', 'p@IPR036157'), ('h@IPR027409', 'p@IPR029054'), ('h@IPR027409', 'p@GO:0016020'), ('h@IPR027409', 'p@GO:0016021'), ('h@IPR027409', 'p@IPR036157'), ('h@GO:0061077', 'p@IPR029054'), ('h@GO:0061077', 'p@GO:0016020'), ('h@GO:0061077', 'p@GO:0016021'), ('h@GO:0061077', 'p@IPR036157'), ('h@GO:0005515', 'p@IPR029054'), ('h@GO:0005515', 'p@GO:0016020'), ('h@GO:0005515', 'p@GO:0016021'), ('h@GO:0005515', 'p@IPR036157'), ('h@GO:0003723', 'p@IPR029054'), ('h@GO:0003723', 'p@GO:0016020'), ('h@GO:0003723', 'p@GO:0016021'), ('h@GO:0003723', 'p@IPR036157'), ('h@IPR027410', 'p@IPR029054'), ('h@IPR027410', 'p@GO:0016020'), ('h@IPR027410', 'p@GO:0016021'), ('h@IPR027410', 'p@IPR036157'), ('h@GO:0005874', 'p@IPR029054'), ('h@GO:0005874', 'p@GO:0016020'), ('h@GO:0005874', 'p@GO:0016021'), ('h@GO:0005874', 'p@IPR036157'), ('h@IPR002194', 'p@IPR029054'), ('h@IPR002194', 'p@GO:0016020'), ('h@

In [33]:
b2 = set().union(*a2)

In [34]:
len(b2)

3242

Comparing the `union` method on lists vs sets.

In [35]:
%timeit set().union(*ppi_df['annotation_pairs_set'])

806 ms ± 51.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
pairs = set().union(*ppi_df['annotation_pairs'])
pairs_set = set().union(*ppi_df['annotation_pairs_set'])

In [37]:
list(pairs)[0:10]

[('h@GO:0071014', 'p@GO:0008270'),
 ('h@GO:0060707', 'p@IPR029052'),
 ('h@GO:0097421', 'p@GO:0045892'),
 ('h@GO:0032607', 'p@GO:1990219'),
 ('h@GO:0030260', 'p@GO:0016020'),
 ('h@IPR005755', 'p@GO:0006230'),
 ('h@GO:0010389', 'p@IPR038765'),
 ('h@GO:0043970', 'p@GO:0003700'),
 ('h@GO:0005267', 'p@GO:0000209'),
 ('h@GO:0140067', 'p@IPR027417')]

In [38]:
list(pairs_set)[0:10]

[('h@GO:0071014', 'p@GO:0008270'),
 ('h@GO:0060707', 'p@IPR029052'),
 ('h@GO:0097421', 'p@GO:0045892'),
 ('h@GO:0032607', 'p@GO:1990219'),
 ('h@GO:0030260', 'p@GO:0016020'),
 ('h@IPR005755', 'p@GO:0006230'),
 ('h@GO:0010389', 'p@IPR038765'),
 ('h@GO:0043970', 'p@GO:0003700'),
 ('h@GO:0005267', 'p@GO:0000209'),
 ('h@GO:0140067', 'p@IPR027417')]

This `union` method seems to work fine on lists (instead of sets).

In [39]:
sorted(pairs) == sorted(list(pairs_set))

True

In [40]:
len(pairs)

783261

In [41]:
sys.getsizeof(pairs) 

33554656

In [42]:
get_size(pairs)

96488097

In [43]:
sys.getsizeof(list(pairs)) 

7049456

In [44]:
get_size(list(pairs))

69982897

In [45]:
np.array(list(pairs)).nbytes

75193056

In [46]:
sys.getsizeof(np.array(list(pairs)))

75193168

In [47]:
get_size(np.array(list(pairs)))

75193546

## Create all possible pairs

In [48]:
### DON'T DO THIS => high memory usage and unnecessary pairs!

In [ ]:
%timit list(itertools.combinations(set().union(*ppi_df[columns].values.ravel('K')), 2))

In [ ]:
pairs_all = list(itertools.combinations(set().union(*ppi_df[columns].values.ravel('K')), 2))

In [17]:
len(pairs_all)

61654960

In [20]:
sys.getsizeof(pairs_all) 

509119928

In [21]:
del pairs_all

In [47]:
list(pairs)[0]

('GO:0000209', 'GO:0060326')

## Merging annotations

In [49]:
ppi_df[columns]

,annotations_A,annotations_B
0,"{h@GO:0071987, h@GO:1904874, h@GO:0005737, h@I...","{p@IPR036157, p@GO:0016021, p@GO:0016020, p@IP..."
1,"{h@GO:0046872, h@GO:0001618, h@GO:0016032, h@G...","{p@GO:0008270, p@GO:0070491, p@GO:0006351, p@G..."
2,"{h@GO:0016779, h@GO:0016032, h@GO:0019901, h@G...","{p@GO:0003678, p@GO:0034214, p@GO:0006260, p@G..."
3,"{h@GO:0042995, h@GO:0006417, h@GO:0015934, h@G...","{p@IPR014900, p@GO:0006351, p@IPR007031, p@GO:..."
4,"{h@GO:0005681, h@GO:0097157, h@GO:0016363, h@G...","{p@IPR004966, p@GO:0003746, p@GO:0019083, p@GO..."
5,"{h@GO:0005681, h@GO:0097157, h@GO:0016363, h@G...","{p@IPR014900, p@GO:0006351, p@IPR007031, p@GO:..."
6,"{h@GO:0045444, h@GO:0003677, h@GO:0003714, h@G...","{p@GO:0006260, p@GO:0004003, p@IPR014000, p@IP..."
7,"{h@GO:0006357, h@GO:0003677, h@GO:0016607, h@G...","{p@GO:0006260, p@GO:0004003, p@IPR014000, p@IP..."
8,"{h@GO:0045444, h@GO:0003677, h@GO:0003714, h@G...","{p@GO:0003678, p@GO:0034214, p@GO:0006260, p@G..."
9,"{h@GO:0042405, h@GO:0016032, h@GO:0006511, h@I...","{p@GO:0006260, p@GO:0004003, p@IPR014000, p@IP..."


In [50]:
%timeit ppi_df[columns].apply(lambda x: set().union(*x), axis=1)

354 ms ± 11.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
ppi_df[columns].apply(lambda x: set().union(*x), axis=1).loc[0]

{'h@GO:0000166',
 'h@GO:0003723',
 'h@GO:0005515',
 'h@GO:0005524',
 'h@GO:0005737',
 'h@GO:0005829',
 'h@GO:0005832',
 'h@GO:0005874',
 'h@GO:0006457',
 'h@GO:0006458',
 'h@GO:0032212',
 'h@GO:0050821',
 'h@GO:0051082',
 'h@GO:0061077',
 'h@GO:0070062',
 'h@GO:0071987',
 'h@GO:1904851',
 'h@GO:1904871',
 'h@GO:1904874',
 'h@IPR002194',
 'h@IPR002423',
 'h@IPR012722',
 'h@IPR017998',
 'h@IPR027409',
 'h@IPR027410',
 'h@IPR027413',
 'p@GO:0016020',
 'p@GO:0016021',
 'p@IPR029054',
 'p@IPR036157'}

In [13]:
merged_annotations = pairwise.merge_annotations(ppi_df, columns)

In [53]:
merged_annotations[0]

{'h@GO:0000166',
 'h@GO:0003723',
 'h@GO:0005515',
 'h@GO:0005524',
 'h@GO:0005737',
 'h@GO:0005829',
 'h@GO:0005832',
 'h@GO:0005874',
 'h@GO:0006457',
 'h@GO:0006458',
 'h@GO:0032212',
 'h@GO:0050821',
 'h@GO:0051082',
 'h@GO:0061077',
 'h@GO:0070062',
 'h@GO:0071987',
 'h@GO:1904851',
 'h@GO:1904871',
 'h@GO:1904874',
 'h@IPR002194',
 'h@IPR002423',
 'h@IPR012722',
 'h@IPR017998',
 'h@IPR027409',
 'h@IPR027410',
 'h@IPR027413',
 'p@GO:0016020',
 'p@GO:0016021',
 'p@IPR029054',
 'p@IPR036157'}

In [54]:
get_size(merged_annotations)

26553111

In [55]:
get_size(merged_annotations.map(list))

7363991

go_dict recursive_children already supplies a set, so no need to convert this back.

In [56]:
go_dict['GO:0005874'].recursive_children

{'GO:0000235',
 'GO:0005827',
 'GO:0005828',
 'GO:0005876',
 'GO:0005879',
 'GO:0005880',
 'GO:0005881',
 'GO:0010005',
 'GO:0020025',
 'GO:0033289',
 'GO:0035371',
 'GO:0036449',
 'GO:0055028',
 'GO:0061673',
 'GO:0097649',
 'GO:0097650',
 'GO:1901588',
 'GO:1903754',
 'GO:1904511',
 'GO:1905721',
 'GO:1905759',
 'GO:1905760',
 'GO:1990498',
 'GO:1990537',
 'GO:1990574',
 'GO:1990719',
 'GO:1990720',
 'GO:1990752',
 'GO:1990941'}

## Propagate pairs

Store propagated pairs as lists.

In [57]:
%timeit [len(pair) for pair in pairs]
%timeit [len(pair) for pair in list(pairs)]

194 ms ± 23.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
224 ms ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
%timeit for pair in list(pairs)[:1000]:    pairwise._propagate_pair(pair, go_dict)

115 ms ± 8.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Store propagated pairs as sets

In [14]:
def _propagate_pair_set(pair, go_dict):
    # create empty list to store the propagated (child) annotations for the two parent annotations in the pair (in same order as original pair)
    propagated_pair = []
    # for both annotations in the pair, propagate through GO hierarchy
    for term in pair:
        # only for GO terms, not IPR
        if 'GO' in term:
            prefix = term[:2]
            go_object = go_dict.get(term[2:])
            # append original annotation if it can't be found in GO dict
            if not go_object:
                propagated_pair.append({term})
            else:
                # store all child terms of parent term in a list
                child_terms = [
                    prefix + i for i in go_object.recursive_children
                ]
                # add parent term itself and remove duplicates
                propagation_set = set(child_terms) | set([term])
                # add propagated annotations to storage list
                propagated_pair.append(propagation_set)
        else:
            # store original term if it's not a GO term
            propagated_pair.append({term})

    # convert the length-2 list of annotation lists (1 for each parent annotation) to a tuple and store in dictionary
    # e.g. ('h@GO:0060384', 'p@GO:0016787') -> (['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'], ['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'])
    return tuple(propagated_pair)

In [45]:
%timeit for pair in list(pairs)[:1000]:    _propagate_pair_set(pair, go_dict)

85 ms ± 5.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Store propagated pairs as lists

In [15]:
def _propagate_pair_list1(pair, go_dict):
    # create empty list to store the propagated (child) annotations for the two parent annotations in the pair (in same order as original pair)
    propagated_pair = []
    # for both annotations in the pair, propagate through GO hierarchy
    for term in pair:
        # only for GO terms, not IPR
        if 'GO' in term:
            prefix = term[:2]
            go_object = go_dict.get(term[2:])
            # append original annotation if it can't be found in GO dict
            if not go_object:
                propagated_pair.append([term])
            else:
                # store all child terms of parent term in a list
                child_terms = [
                    prefix + i for i in go_object.recursive_children
                ]
                # add parent term
                child_terms.append(term)
                # remove duplicates, convert to list and sort
                propagation_list = list(set(child_terms))
                # add propagated annotations to storage list
                propagated_pair.append(propagation_list)
        else:
            # store original term if it's not a GO term
            propagated_pair.append([term])

    # convert the length-2 list of annotation lists (1 for each parent annotation) to a tuple and store in dictionary
    # e.g. ('h@GO:0060384', 'p@GO:0016787') -> (['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'], ['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'])
    return tuple(propagated_pair)

In [46]:
%timeit for pair in list(pairs)[:1000]:    _propagate_pair_list1(pair, go_dict)

90.4 ms ± 8.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Store propagated pairs as sorted lists

In [16]:
def _propagate_pair_list2(pair, go_dict):
    # create empty list to store the propagated (child) annotations for the two parent annotations in the pair (in same order as original pair)
    propagated_pair = []
    # for both annotations in the pair, propagate through GO hierarchy
    for term in pair:
        # only for GO terms, not IPR
        if 'GO' in term:
            prefix = term[:2]
            go_object = go_dict.get(term[2:])
            # append original annotation if it can't be found in GO dict
            if not go_object:
                propagated_pair.append([term])
            else:
                # store all child terms of parent term in a list
                child_terms = [
                    prefix + i for i in go_object.recursive_children
                ]
                # add parent term
                child_terms.append(term)
                # remove duplicates, convert to list and sort
                propagation_list = sorted(set(child_terms))
                # add propagated annotations to storage list
                propagated_pair.append(propagation_list)
        else:
            # store original term if it's not a GO term
            propagated_pair.append([term])

    # convert the length-2 list of annotation lists (1 for each parent annotation) to a tuple and store in dictionary
    # e.g. ('h@GO:0060384', 'p@GO:0016787') -> (['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'], ['h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'])
    return tuple(propagated_pair)

In [47]:
%timeit for pair in list(pairs)[:1000]:    _propagate_pair_list2(pair, go_dict)

140 ms ± 7.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Propagated pairs dictionary memory requirements

The propagated dict can take up to 11-25 GB of RAM when it is structured as a dictionary of tuples pointing to sets.

In [17]:
def _propagate_pairs_set(pairs, go_dict):
    # create empty dictionary to store the two sets of propagated annotations
    propagated_pairs = {}
    # propagated_pairs = []

    # loop through sets of annotation pairs
    for pair in list(pairs):
        # create empty list to store the propagated (child) annotations for the two parent annotations in the pair (in same order as original pair)
        new_pair = []
        # for both annotations in the pair, propagate through GO hierarchy
        for term in pair:
            # only for GO terms, not IPR
            if 'GO' in term:
                prefix = term[:2]
                go_object = go_dict.get(term[2:])
                # append original annotation if it can't be found in GO dict
                if not go_object:
                    # store original term if no GO info available
                    new_pair.append({term})
                else:
                    # store all child terms of parent term in a list
                    child_terms = [
                        prefix + i for i in go_object.recursive_children
                    ]
                    # add parent term itself and remove duplicates
                    propagation_set = set(child_terms) | set([term])
                    # add propagated annotations to storage list
                    new_pair.append(propagation_set)
            else:
                # store original term if it's not a GO term
                new_pair.append({term})

        # convert the length-2 list of annotation lists (1 for each parent annotation) to a tuple and store in dictionary
        # e.g. ('h@GO:0060384', 'p@GO:0016787') -> ({'h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'}, {'h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'})
        propagated_pairs[pair] = tuple(new_pair)

    return propagated_pairs

In [53]:
pairs_dict = _propagate_pairs_set(list(pairs)[:100], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100 / 10**6)

3168777
24819.794417970003


In [87]:
pairs_dict = _propagate_pairs_set(list(pairs)[:100000], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100000 / 10**6)

2166206902
16967.05384267422


Previous values...due to random order inside pairs set?

In [120]:
pairs_dict = pairwise._propagate_pairs(list(pairs)[:100], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100 / 10**6)

1452605
11377.68844905


In [123]:
pairs_dict = _propagate_pairs_set(list(pairs)[:100000], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100000 / 10**6)

2192436790
17172.5023257219


In [112]:
get_size(pairs_dict[(('h@GO:0006911', 'p@GO:0006355'))])

70411

A set takes almost double the memory of a list.

In [60]:
get_size({'h@GO:0000166', 'h@GO:0003723', 'h@GO:0005515', 'h@GO:0005524', 'h@GO:0005737', 'h@GO:0005829', 'h@GO:0005832', 'h@GO:0005874', 'h@GO:0006457', 'h@GO:0006458', 'h@GO:0032212', 'h@GO:0050821', 'h@GO:0051082', 'h@GO:0061077', 'h@GO:0070062', 'h@GO:0071987', 'h@GO:1904851', 'h@GO:1904871', 'h@GO:1904874', 'h@IPR002194', 'h@IPR002423', 'h@IPR012722', 'h@IPR017998', 'h@IPR027409', 'h@IPR027410', 'h@IPR027413'
         })

3851

In [59]:
get_size(list({'h@GO:0000166', 'h@GO:0003723', 'h@GO:0005515', 'h@GO:0005524', 'h@GO:0005737', 'h@GO:0005829', 'h@GO:0005832', 'h@GO:0005874', 'h@GO:0006457', 'h@GO:0006458', 'h@GO:0032212', 'h@GO:0050821', 'h@GO:0051082', 'h@GO:0061077', 'h@GO:0070062', 'h@GO:0071987', 'h@GO:1904851', 'h@GO:1904871', 'h@GO:1904874', 'h@IPR002194', 'h@IPR002423', 'h@IPR012722', 'h@IPR017998', 'h@IPR027409', 'h@IPR027410', 'h@IPR027413'
         }))

1923

In [18]:
def _propagate_pairs_list(pairs, go_dict):
    # create empty dictionary to store the two sets of propagated annotations
    propagated_pairs = {}
    # propagated_pairs = []

    # loop through sets of annotation pairs
    for pair in list(pairs):
        # create empty list to store the propagated (child) annotations for the two parent annotations in the pair (in same order as original pair)
        new_pair = []
        # for both annotations in the pair, propagate through GO hierarchy
        for term in pair:
            # only for GO terms, not IPR
            if 'GO' in term:
                prefix = term[:2]
                go_object = go_dict.get(term[2:])
                # append original annotation if it can't be found in GO dict
                if not go_object:
                    # store original term if no GO info available
                    new_pair.append([term])
                else:
                    # store all child terms of parent term in a list
                    child_terms = [
                        prefix + i for i in go_object.recursive_children
                    ]
                    # add parent term itself and remove duplicates
                    propagation_set = list(set(child_terms) | set([term]))
                    # add propagated annotations to storage list
                    new_pair.append(propagation_set)
            else:
                # store original term if it's not a GO term
                new_pair.append([term])

        # convert the length-2 list of annotation lists (1 for each parent annotation) to a tuple and store in dictionary
        # e.g. ('h@GO:0060384', 'p@GO:0016787') -> ({'h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'}, {'h@GO:0098546', 'h@GO:0030553', 'h@GO:0035438', 'h@GO:0030552', 'h@GO:0061507'})
        propagated_pairs[pair] = tuple(new_pair)

    return propagated_pairs

In [55]:
pairs_dict = _propagate_pairs_list(list(pairs)[:100], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100 / 10**6)

1963849
15382.06331589


In [62]:
pairs_dict = _propagate_pairs_list(list(pairs)[:100000], go_dict)
print(get_size(pairs_dict))
print(len(pairs) * get_size(pairs_dict) / 100000 / 10**6)

1364941334
10691.05314210174


In [20]:
del pairs_dict

NameError: name 'pairs_dict' is not defined

**Conclusion: storing the propagated pairs as lists is a major reduction in size, however, doing this forces a `set()` operation to take place during the pairwise association mining procedure.**

## Pairwise assoc
Comparing the performance of pairwise association mining when

1. pre-computing the list of propagated pairs
2. computing the list as an iterator
3. computing the propagated pair on the fly inside the loop
4. Using set objects as input
5. Converting list objects to sets inside the loop

### 1 pre-compute propagated pairs

#### Pre-compute + pairwise_dict with sets + no convert to set during loop

In [19]:
def count_presences_propagated_pre(pair_prop, interaction_dataframe, column_A,
                               column_B, merged_annotations):
    set_A = pair_prop[0]
    set_B = pair_prop[1]

    pair_count = np.sum(
        merged_annotations.map(
            lambda x: not x.isdisjoint(set_A) and not x.isdisjoint(set_B)
        ))
    label_one_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: not x.isdisjoint(set_A)
    ) & interaction_dataframe[column_B].map(
        lambda x: x.isdisjoint(set_B)))
    label_two_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: x.isdisjoint(set_A)
    ) & interaction_dataframe[column_B].map(
        lambda x: not x.isdisjoint(set_B)))
    label_one_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(set_A)))
    label_two_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(set_B)))
    absent_count = np.sum(
        merged_annotations.map(lambda x: x.isdisjoint(set_A)) &
        merged_annotations.map(lambda x: x.isdisjoint(set_B)))
    total_count = interaction_dataframe.shape[0]
    return pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count

In [21]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
pairs_dict = _propagate_pairs_set(list(pairs)[:100], go_dict)
counter = 0
for pair, propagated_pair in pairs_dict.items():
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
    propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

7.35 s ± 492 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Pre-compute + pairwise_dict with lists + convert to set during loop

In [24]:
def count_presences_propagated_pre_set(pair_prop, interaction_dataframe, column_A,
                               column_B, merged_annotations):
    set_A = set(pair_prop[0])
    set_B = set(pair_prop[1])

    pair_count = np.sum(
        merged_annotations.map(
            lambda x: not x.isdisjoint(set_A) and not x.isdisjoint(set_B)
        ))
    label_one_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: not x.isdisjoint(set_A)
    ) & interaction_dataframe[column_B].map(
        lambda x: x.isdisjoint(set_B)))
    label_two_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: x.isdisjoint(set_A)
    ) & interaction_dataframe[column_B].map(
        lambda x: not x.isdisjoint(set_B)))
    label_one_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(set_A)))
    label_two_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(set_B)))
    absent_count = np.sum(
        merged_annotations.map(lambda x: x.isdisjoint(set_A)) &
        merged_annotations.map(lambda x: x.isdisjoint(set_B)))
    total_count = interaction_dataframe.shape[0]
    return pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count

In [25]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
pairs_dict = _propagate_pairs_list(list(pairs)[:100], go_dict)
counter = 0
for pair, propagated_pair in pairs_dict.items():
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
#     propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

7.06 s ± 100 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**There's next to no cost (~ -0.2 to + 1 second on multiple tries) to converting lists to sets inside the mining function, so it is better to store lists inside the propagated dictionary to reduce memory requirements.**

In [26]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)
counter = 0
for pair, propagated_pair in pairs_dict.items():
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
    propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

1min 7s ± 4.35 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Splitting up the generation and mining loop.**

In [29]:
%%timeit
pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)

139 ms ± 8.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)

In [32]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
counter = 0
for pair, propagated_pair in pairs_dict.items():
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
    propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

1min 22s ± 4.75 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


**More detailed profiling of time spent during function...**

In [33]:
%%prun
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)
counter = 0
for pair, propagated_pair in pairs_dict.items():
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
    propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

```
        271579452 function calls (271574452 primitive calls) in 113.221 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
132513401   43.296    0.000   43.296    0.000 {method 'isdisjoint' of 'set' objects}
     9000   29.748    0.003   92.417    0.010 {pandas._libs.lib.map_infer}
    27000    6.931    0.000    6.931    0.000 {method 'copy' of 'numpy.ndarray' objects}
 14685000    2.277    0.000    7.741    0.000 <ipython-input-24-fe4583da8d50>:8(<lambda>)
 14685000    2.202    0.000    6.984    0.000 <ipython-input-24-fe4583da8d50>:11(<lambda>)
 14685000    2.189    0.000    8.508    0.000 <ipython-input-24-fe4583da8d50>:21(<lambda>)
 14685000    2.186    0.000    7.480    0.000 <ipython-input-24-fe4583da8d50>:19(<lambda>)
 14685000    2.130    0.000    4.719    0.000 <ipython-input-24-fe4583da8d50>:17(<lambda>)
 14685000    2.115    0.000    8.435    0.000 <ipython-input-24-fe4583da8d50>:24(<lambda>)
 14685000    2.111    0.000    7.346    0.000 <ipython-input-24-fe4583da8d50>:23(<lambda>)
 14685000    2.092    0.000    6.776    0.000 <ipython-input-24-fe4583da8d50>:15(<lambda>)
 14685000    2.071    0.000    4.680    0.000 <ipython-input-24-fe4583da8d50>:13(<lambda>)
     7000    1.624    0.000   79.241    0.011 series.py:2904(map)
     6000    0.712    0.000    0.943    0.000 missing.py:189(_isna_ndarraylike)
     1000    0.542    0.001  109.164    0.109 <ipython-input-24-fe4583da8d50>:1(count_presences_propagated_pre_set)
    14834    0.537    0.000    0.537    0.000 _discrete_distns.py:351(_logpmf)
    45327    0.496    0.000    0.496    0.000 {method 'reduce' of 'numpy.ufunc' objects}
  1149915    0.433    0.000    0.814    0.000 {built-in method builtins.isinstance}
    13831    0.405    0.000    2.556    0.000 _distn_infrastructure.py:2811(pmf)
    33000    0.399    0.000    2.135    0.000 series.py:166(__init__)
   345129    0.290    0.000    0.290    0.000 {built-in method numpy.core.multiarray.array}
    21000    0.258    0.000    8.070    0.000 internals.py:3500(apply)
   434506    0.210    0.000    0.248    0.000 {built-in method builtins.getattr}
    16834    0.210    0.000    0.969    0.000 _distn_infrastructure.py:549(<listcomp>)
    15079    0.190    0.000    0.190    0.000 _discrete_distns.py:344(_argcheck)
    39000    0.167    0.000    0.318    0.000 generic.py:4378(__setattr__)
    33000    0.142    0.000    0.655    0.000 internals.py:4639(__init__)
    15000    0.141    0.000    7.432    0.000 internals.py:577(_astype)
    63336    0.134    0.000    0.759    0.000 function_base.py:2299(extract)
...
```

In [34]:
%load_ext line_profiler
%load_ext heat

In [36]:
def fast(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations):
    results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
    pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)
    counter = 0
    for pair, propagated_pair in pairs_dict.items():
        counter += 1
        if counter % 10000 == 0:
            print('Processed {} lines out of {}'.format(
                counter, len(pairs)))
        # convert lists of annotations to sets in order to speed up comparisons
#         propagated_pair = list(set(i) for i in propagated_pair)
        pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
            propagated_pair, ppi_df, column_A, column_B,
            merged_annotations)
        pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
%lprun -f fast fast(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations)

```
Timer unit: 1e-06 s

Total time: 113.407 s
File: <ipython-input-35-9abf4a21e976>
Function: fast at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def fast(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations):
     2         1          3.0      3.0      0.0      results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
     3         1     172288.0 172288.0      0.2      pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)
     4         1          1.0      1.0      0.0      counter = 0
     5      1001       3143.0      3.1      0.0      for pair, propagated_pair in pairs_dict.items():
     6      1000        988.0      1.0      0.0          counter += 1
     7      1000       1096.0      1.1      0.0          if counter % 10000 == 0:
     8                                                       print('Processed {} lines out of {}'.format(
     9                                                           counter, len(pairs)))
    10                                                   # convert lists of annotations to sets in order to speed up comparisons
    11                                           #         propagated_pair = list(set(i) for i in propagated_pair)
    12      1000       1228.0      1.2      0.0          pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
    13      1000        775.0      0.8      0.0              propagated_pair, ppi_df, column_A, column_B,
    14      1000  109179618.0 109179.6     96.3              merged_annotations)
    15      1000    4047975.0   4048.0      3.6          pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
    ```

```
Timer unit: 1e-06 s

Total time: 91.7701 s
File: <ipython-input-68-d38c677a9b5a>
Function: fast at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def fast(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations):
     2         1          2.0      2.0      0.0      results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
     3         1     154876.0 154876.0      0.2      pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)
     4         1          2.0      2.0      0.0      counter = 0
     5      1001       2223.0      2.2      0.0      for pair, propagated_pair in pairs_dict.items():
     6      1000        832.0      0.8      0.0          counter += 1
     7      1000        939.0      0.9      0.0          if counter % 10000 == 0:
     8                                                       print('Processed {} lines out of {}'.format(
     9                                                           counter, len(pairs)))
    10                                                   # convert lists of annotations to sets in order to speed up comparisons
    11                                           #         propagated_pair = list(set(i) for i in propagated_pair)
    12      1000        885.0      0.9      0.0          pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
    13      1000        701.0      0.7      0.0              propagated_pair, ppi_df, column_A, column_B,
    14      1000   88386176.0  88386.2     96.3              merged_annotations)
    15      1000    3223449.0   3223.4      3.5          pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
    
```

---
Old version still converted the propagated pair sets to list, which is no longer required.

```
Timer unit: 1e-06 s

Total time: 95.401 s
File: <ipython-input-65-09a22af8a058>
Function: fast at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def fast(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations):
     2         1          2.0      2.0      0.0      results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
     3         1     169316.0 169316.0      0.2      pairs_dict = _propagate_pairs_list(list(pairs)[:1000], go_dict)
     4         1          2.0      2.0      0.0      counter = 0
     5      1001       8571.0      8.6      0.0      for pair, propagated_pair in pairs_dict.items():
     6      1000       1013.0      1.0      0.0          counter += 1
     7      1000       1125.0      1.1      0.0          if counter % 10000 == 0:
     8                                                       print('Processed {} lines out of {}'.format(
     9                                                           counter, len(pairs)))
    10                                                   # convert lists of annotations to sets in order to speed up comparisons
    11      1000      22910.0     22.9      0.0          propagated_pair = list(set(i) for i in propagated_pair)
    12      1000       1061.0      1.1      0.0          pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated(
    13      1000        730.0      0.7      0.0              propagated_pair, ppi_df, column_A, column_B,
    14      1000   91837815.0  91837.8     96.3              merged_annotations)
    15      1000    3358479.0   3358.5      3.5          pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
    ```

### 3 on the fly calculation of propagated pairs

On the fly propagation is an order of magnitude slower (seconds to 1 minute)! 

However, if the separate propagation step is counted, the difference vanishes...

In [37]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
counter = 0
for pair in list(pairs)[:1000]:
        propagated_pair = _propagate_pair_list1(pair, go_dict)
        counter += 1
        if counter % 10000 == 0:
            print('Processed {} lines out of {}'.format(
                counter, len(pairs)))
        propagated_pair = list(set(i) for i in propagated_pair)
        pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
            propagated_pair, ppi_df, column_A, column_B,
            merged_annotations)

        pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

1min 19s ± 2.79 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
def mem(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations):
    results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
    counter = 0
    for pair in list(pairs)[:1000]:
            propagated_pair = _propagate_pair_list1(pair, go_dict)
            counter += 1
            if counter % 10000 == 0:
                print('Processed {} lines out of {}'.format(
                    counter, len(pairs)))
            propagated_pair = list(set(i) for i in propagated_pair)
            pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
                propagated_pair, ppi_df, column_A, column_B,
                merged_annotations)

            pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
%lprun -f mem mem(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations)

```
Timer unit: 1e-06 s

Total time: 87.832 s
File: <ipython-input-38-3dfb1a5cc234>
Function: mem at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def mem(pairs, go_dict, ppi_df, column_A, column_B, merged_annotations):
     2         1          4.0      4.0      0.0      results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
     3         1          1.0      1.0      0.0      counter = 0
     4      1001      44837.0     44.8      0.1      for pair in list(pairs)[:1000]:
     5      1000      91699.0     91.7      0.1              propagated_pair = _propagate_pair_list1(pair, go_dict)
     6      1000       1026.0      1.0      0.0              counter += 1
     7      1000       1075.0      1.1      0.0              if counter % 10000 == 0:
     8                                                           print('Processed {} lines out of {}'.format(
     9                                                               counter, len(pairs)))
    10      1000       9900.0      9.9      0.0              propagated_pair = list(set(i) for i in propagated_pair)
    11      1000        700.0      0.7      0.0              pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
    12      1000        667.0      0.7      0.0                  propagated_pair, ppi_df, column_A, column_B,
    13      1000   84583709.0  84583.7     96.3                  merged_annotations)
    14                                           
    15      1000    3098399.0   3098.4      3.5              pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
```

### Generator

A generator is equall fast as the other two options, but it doesn't store the object in memory. Clear winner. Note that this also allows the storage of propagated pairs as sets instead of lists again, removing the need to convert them inside the counting function.

In [39]:
def prop_iterator(pairs, go_dict):
    # loop through sets of annotation pairs
    for pair in pairs:
        propagated_pair = _propagate_pair_list1(pair, go_dict)
        yield pair, tuple(propagated_pair)

In [40]:
for pair, propagated_pair in prop_iterator(list(pairs)[:2], go_dict):
    print(pair, propagated_pair)

('h@IPR019035', 'p@GO:0005515') (['h@IPR019035'], ['p@GO:0070699', 'p@GO:0042609', 'p@GO:0031995', 'p@GO:0051373', 'p@GO:0043185', 'p@GO:0045522', 'p@GO:0005067', 'p@GO:0001864', 'p@GO:0019838', 'p@GO:0097020', 'p@GO:0001853', 'p@GO:0019961', 'p@GO:0015631', 'p@GO:0045159', 'p@GO:1902379', 'p@GO:0005143', 'p@GO:0051428', 'p@GO:0031883', 'p@GO:0090736', 'p@GO:0031820', 'p@GO:0031760', 'p@GO:0050733', 'p@GO:0042007', 'p@GO:0072568', 'p@GO:0051530', 'p@GO:0038103', 'p@GO:0031817', 'p@GO:0031695', 'p@GO:0031008', 'p@GO:0070080', 'p@GO:0048273', 'p@GO:0019973', 'p@GO:0031777', 'p@GO:0042324', 'p@GO:0031867', 'p@GO:0001521', 'p@GO:0001097', 'p@GO:0019905', 'p@GO:0033149', 'p@GO:0031819', 'p@GO:0031700', 'p@GO:0043395', 'p@GO:0032034', 'p@GO:0051579', 'p@GO:0070063', 'p@GO:0070016', 'p@GO:1990311', 'p@GO:0070325', 'p@GO:0071794', 'p@GO:0030370', 'p@GO:0031808', 'p@GO:0031800', 'p@GO:0097109', 'p@GO:0019958', 'p@GO:0016220', 'p@GO:0008017', 'p@GO:0140081', 'p@GO:0019966', 'p@GO:0032840', 'p@GO

In [41]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
counter = 0
for pair, propagated_pair in prop_iterator(list(pairs)[:1000], go_dict):
    counter += 1
    if counter % 10000 == 0:
        print('Processed {} lines out of {}'.format(
            counter, len(pairs)))
    # convert lists of annotations to sets in order to speed up comparisons
    propagated_pair = list(set(i) for i in propagated_pair)
    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
        propagated_pair, ppi_df, column_A, column_B,
        merged_annotations)
    pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)

1min 1s ± 123 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
results_dict

{'pmi': {('h@IPR019035', 'p@GO:0005515'): 1.4738220338312717,
  ('h@GO:0006024', 'p@IPR036092'): 0.7615711881446453,
  ('h@IPR024743', 'p@GO:0033668'): 0.31347944287701446,
  ('h@GO:0071782', 'p@GO:0033644'): 1.0267144872604053,
  ('h@GO:0036122', 'p@GO:0030683'): 0.03865604116099738,
  ('h@GO:0032298', 'p@GO:0005524'): 3.060695931687554,
  ('h@GO:0043125', 'p@GO:0030430'): -0.0359569382334961,
  ('h@IPR005937', 'p@GO:1990216'): 0.11025933081541205,
  ('h@GO:0007173', 'p@IPR018272'): 5.028274453995176,
  ('h@GO:0050709', 'p@GO:0000166'): -0.8856521322604716,
  ('h@GO:0007585', 'p@GO:0075342'): 1.4529621233077399,
  ('h@IPR017347', 'p@GO:0033644'): 0.03752124221698825,
  ('h@GO:1903748', 'p@GO:0042025'): -0.3885090019007959,
  ('h@IPR002231', 'p@IPR000662'): 7.100588658811066,
  ('h@GO:0008202', 'p@IPR002475'): -0.3066592479608885,
  ('h@GO:0045084', 'p@GO:0090630'): 3.2416780921043697,
  ('h@GO:0042803', 'p@GO:0061635'): 3.2455552646633548,
  ('h@IPR023123', 'p@IPR038575'): -1.21229429

## Effect of conditional inside versus outside loops

Barely any effect at all.

In [43]:
prop = True
fast = True

In [44]:
%%timeit
results_dict = {'pmi': {},'G': {},'chi2': {},'fisher': {},'phi': {},'min_count': {},'counts': {},'depth': {}}
counter = 0
for pair, propagated_pair in prop_iterator(list(pairs)[:1000], go_dict):
    if prop:
        if fast:
            counter += 1
            if counter % 10000 == 0:
                print('Processed {} lines out of {}'.format(
                    counter, len(pairs)))
            # convert lists of annotations to sets in order to speed up comparisons
            propagated_pair = list(set(i) for i in propagated_pair)
            pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
                propagated_pair, ppi_df, column_A, column_B,
                merged_annotations)
            pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
        else:
            counter += 1
            if counter % 10000 == 0:
                print('Processed {} lines out of {}'.format(
                    counter, len(pairs)))
            # convert lists of annotations to sets in order to speed up comparisons
            propagated_pair = list(set(i) for i in propagated_pair)
            pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
                propagated_pair, ppi_df, column_A, column_B,
                merged_annotations)
            pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)
    else:
        counter += 1
        if counter % 10000 == 0:
            print('Processed {} lines out of {}'.format(
                counter, len(pairs)))
        # convert lists of annotations to sets in order to speed up comparisons
        propagated_pair = list(set(i) for i in propagated_pair)
        pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = count_presences_propagated_pre_set(
            propagated_pair, ppi_df, column_A, column_B,
            merged_annotations)
        pairwise._calc_assoc_measures(pair, results_dict, pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count, go_dict)


55.8 s ± 88.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
ppi_df['annotation_pairs']

0        [(h@GO:0050821, p@IPR036157), (h@GO:0050821, p...
1        [(h@GO:0016032, p@GO:0039650), (h@GO:0016032, ...
2        [(h@GO:0005654, p@GO:0039693), (h@GO:0005654, ...
3        [(h@IPR034278, p@IPR007031), (h@IPR034278, p@I...
4        [(h@GO:0005654, p@GO:0019031), (h@GO:0005654, ...
5        [(h@GO:0005654, p@IPR007031), (h@GO:0005654, p...
6        [(h@GO:0005654, p@IPR001177), (h@GO:0005654, p...
7        [(h@GO:0005654, p@IPR001177), (h@GO:0005654, p...
8        [(h@GO:0005654, p@GO:0039693), (h@GO:0005654, ...
9        [(h@IPR013026, p@IPR001177), (h@IPR013026, p@G...
10       [(h@GO:0005654, p@GO:0039693), (h@GO:0005654, ...
11       [(h@GO:0005654, p@GO:0003697), (h@GO:0005654, ...
12       [(h@GO:0005654, p@GO:0003697), (h@GO:0005654, ...
13       [(h@GO:0031434, p@GO:0016032), (h@GO:0031434, ...
14       [(h@GO:0097421, p@GO:0016032), (h@GO:0097421, ...
15       [(h@GO:0005654, p@GO:0003697), (h@GO:0005654, ...
16       [(h@IPR017305, p@GO:0016032), (h@IPR017305, p@.